In [1]:
import deep_predictor
from collections.abc import Mapping
import torch
import scanpy as sc

Global seed set to 0
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
import scipy

In [3]:
import numpy as np
import pandas as pd

In [4]:
#adata = sc.read_h5ad("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/tree_data/Adata_subset.h5ad")

In [5]:
#adata = sc.read_h5ad("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/adata_segment_subset.h5ad")
path_to_file = "/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/unstructured_data/Mouse_URD_tree_data.csv"
adata = sc.read_csv(path_to_file, delimiter=',').T

In [6]:
annotations = pd.read_csv("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/unstructured_data/Mouse_URD_meta.csv",index_col = ["Unnamed: 0"] )
adata.obs = annotations

/tmp/ipykernel_3288/714144093.py:1: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations = pd.read_csv("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/unstructured_data/Mouse_URD_meta.csv",index_col = ["Unnamed: 0"] )


In [7]:
#adata.write_h5ad("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/unstructured_data/Adata_mouse.h5ad")

In [8]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=50)

In [9]:
adata = adata[np.logical_not(adata.obs["stage"] == 'mixed_gastrulation')]

In [10]:
Time = []
for i in adata.obs["stage"] :
    Time.append(float(i.split("E")[1]))

In [11]:
adata.obs["Time"] = np.array(Time)

/tmp/ipykernel_3288/3102962000.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["Time"] = np.array(Time)


In [12]:
np.unique(adata.obs["celltype"])

array(['Anterior Primitive Streak', 'Cardiomyocytes', 'Caudal Mesoderm',
       'Caudal epiblast', 'Caudal neurectoderm', 'Def. endoderm',
       'Epiblast', 'Forebrain/Midbrain/Hindbrain', 'Gut',
       'Intermediate mesoderm', 'Mesenchyme', 'Mixed mesoderm', 'NMP',
       'Nascent mesoderm', 'Neural crest', 'Notochord', 'PGC',
       'Paraxial mesoderm', 'Pharyngeal mesoderm', 'Primitive Streak',
       'Rostral neurectoderm', 'Somitic mesoderm', 'Spinal cord',
       'Surface ectoderm'], dtype=object)

In [13]:
## Ectoderm  28,29 - 30,31,32,33
ectoderm_celltype = ['Rostral neurectoderm','Caudal neurectoderm','Neural crest','Forebrain/Midbrain/Hindbrain','Spinal cord','Surface ectoderm']

In [14]:
## Mesoderm  9,10 - 11,12,13,14,16
mesoderm_celltype = ['Nascent mesoderm','Mixed mesoderm','Caudal Mesoderm','Paraxial mesoderm','Somitic mesoderm','Pharyngeal mesoderm','Cardiomyocytes']

In [15]:
#Training Data
ectoderm_celltype = ['Neural crest','Forebrain/Midbrain/Hindbrain','Spinal cord','Surface ectoderm']

prog_data_1 = adata[adata.obs['celltype'] == 'Rostral neurectoderm']
random_index = np.random.choice(np.arange(0,np.shape(prog_data_1)[0]),500)
prog_data_1 = prog_data_1[random_index]


prog_data_2 = adata[adata.obs['celltype'] == 'Caudal neurectoderm']
random_index = np.random.choice(np.arange(0,np.shape(prog_data_2)[0]),500)
prog_data_2 = prog_data_2[random_index]

ectoderm_adata = prog_data_1.concatenate(prog_data_2)

for cell in ectoderm_celltype:
    cell_adata = adata[adata.obs['celltype'] == cell]
    random_index = np.random.choice(np.arange(0,np.shape(cell_adata)[0]),250)
    cell_adata = cell_adata[random_index]
    ectoderm_adata = ectoderm_adata.concatenate(cell_adata)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observatio

In [16]:
#Training Data

#Training Data
mesoderm_celltype = ['Paraxial mesoderm','Somitic mesoderm','Pharyngeal mesoderm','Cardiomyocytes']

prog_data_1 = adata[adata.obs['celltype'] == 'Nascent mesoderm']
random_index = np.random.choice(np.arange(0,np.shape(prog_data_1)[0]),500)
prog_data_1 = prog_data_1[random_index]


prog_data_2 = adata[adata.obs['celltype'] == 'Mixed mesoderm']
random_index = np.random.choice(np.arange(0,np.shape(prog_data_2)[0]),500)
prog_data_2 = prog_data_2[random_index]

mesoderm_adata = prog_data_1.concatenate(prog_data_2)

for cell in mesoderm_celltype:
    cell_adata = adata[adata.obs['celltype'] == cell]
    random_index = np.random.choice(np.arange(0,np.shape(cell_adata)[0]),250)
    cell_adata = cell_adata[random_index]
    mesoderm_adata = mesoderm_adata.concatenate(cell_adata)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observatio

In [17]:
ectoderm_adata.obs["data"] = (np.logical_not((ectoderm_adata.obs["celltype"] == 'Rostral neurectoderm') | (ectoderm_adata.obs["celltype"] == 'Caudal neurectoderm'))).astype('uint8') 
mesoderm_adata.obs["data"] = (np.logical_not((mesoderm_adata.obs["celltype"] == 'Mixed mesoderm') | (mesoderm_adata.obs["celltype"] == 'Nascent mesoderm'))).astype('uint8') 

In [18]:
ectoderm_adata.obs["lineage"] = "ecto"
mesoderm_adata.obs["lineage"] = "meso"

In [19]:
Adata_subset = ectoderm_adata.concatenate(mesoderm_adata)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [22]:
Adata_subset.obs["endo_trajectoryName"]=0
Adata_subset.obs["endo_gutCluster"]=0

In [23]:
Adata_subset.write_h5ad("/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/unstructured_data/Adata_mouse_subset.h5ad")

In [27]:
zebra_obj = deep_predictor.DeepPredictor(mesoderm_adata,latent_dim=100,hidden_layers=[800,800],workers=8,parameters=[0,0,1,0,1],likelihood="nb",batch_size=100)

In [28]:
zebra_obj.train(epochs=150)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | VAEmodel | 26.3 M
-----------------------------------
26.3 M    Trainable params
0         Non-trainable params
26.3 M    Total params
105.340   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=150` reached.


In [29]:
zebra_obj.save_model(path="/home/gaurav/Gaurav/Berlin/Paper/Pijuan_sala_data/unstructured_data/models/Mesoderm_trained_unstruct_1.pt")